In [1]:
from HANEmotionAnalyze import *
import torch
import torch.nn as nn
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json
import thulac
thulac = thulac.thulac()

Model loaded succeed


In [2]:
# import torch.functional as F
# 
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[0, 1,2,4,5],[1, 4,3,2,9]])
print(embedding(input))
embedding2 = nn.Embedding(10, 3, padding_idx=2)
# input = torch.LongTensor([[1,2,0,5, 6,7,8,9]])
print(embedding2(input))
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding3 = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
print(embedding3(input))
input = torch.LongTensor([0])
print(embedding3(input))



tensor([[[ 0.5330,  0.1214,  0.5117],
         [ 0.6444,  0.2468,  1.7833],
         [-1.9501, -1.5159,  2.1440],
         [-0.1257,  0.6628, -1.1044],
         [-1.1152,  0.3954,  0.6023]],

        [[ 0.6444,  0.2468,  1.7833],
         [-0.1257,  0.6628, -1.1044],
         [-0.1768,  0.3839,  0.3991],
         [-1.9501, -1.5159,  2.1440],
         [ 1.1567, -0.8254,  0.0972]]], grad_fn=<EmbeddingBackward>)
tensor([[[ 1.2689,  0.9966,  0.1185],
         [ 0.8692,  0.6171, -0.2417],
         [ 0.0000,  0.0000,  0.0000],
         [-0.4652, -0.9552, -1.0436],
         [ 2.2206,  0.8936,  0.4039]],

        [[ 0.8692,  0.6171, -0.2417],
         [-0.4652, -0.9552, -1.0436],
         [-1.0174, -0.1631, -0.3047],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.2563,  0.5658, -0.5279]]], grad_fn=<EmbeddingBackward>)
tensor([[4.0000, 5.1000, 6.3000]])
tensor([[1.0000, 2.3000, 3.0000]])


## 读取词向量
建立词语列表

In [ ]:
from gensim.models import KeyedVectors, Word2Vec

# file = '../../PretrainedData/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
file = '../../DataSets/Word2Vect/xingrong_50_thulac/word2vect_50_w5.model'
word2vec = Word2Vec.load(file)
# word2vec = KeyedVectors.load_word2vec_format(file, binary=False)
word2vec.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar
word2vec.vector_size

In [ ]:
print(word2vec)
# print(word2vec.wv.vocab)
# print(len(word2vec.index2word))
print(len(word2vec.wv.index2word))
print(word2vec.wv.index2word[0])
print(word2vec.wv.index2word[1])
print(word2vec.wv.index2word[2])
print(word2vec.wv.index2word[1522])
print(word2vec.wv.index2entity[1522])
print(word2vec.similar_by_word('中国'))
print(word2vec.similar_by_word('天才'))
print(word2vec.wv)
print('word2vec.wv.vocab ---- >', word2vec.wv.vocab)
print(word2vec.wv.index2word)


In [ ]:
wordEmbedding = [word2vec.wv[word]  for word in word2vec.wv.index2word]
word2index = { word:i for i, word in enumerate(word2vec.wv.index2word)}
print(wordEmbedding[:10])
print(word2index['中国'])
print(word2index['天才'])

In [ ]:
#创建模型
wordEmbedding = torch.FloatTensor(wordEmbedding)
num_embeddings = len(word2vec.wv.index2word)
model = HAN(num_embeddings, embedding_dim=word2vec.wv.vector_size)
print(model)
model.embed.from_pretrained(wordEmbedding)


## 读取训练数据

In [ ]:
def compute_ngrams(word, num_min = 1, num_max = 3):
    ngrams =[]
    for ngram_length in range(num_min, min(len(word), num_max) + 1):
        for i in range(len(word) - ngram_length + 1):
            # print(i, i + ngram_length)
            ngrams.append(word[i : i + ngram_length])
    # print(ngrams)
    return list(set(ngrams))

print(compute_ngrams('you'))
print(compute_ngrams('I Think'))
print(compute_ngrams('中华人民共和国万岁'))

In [ ]:
# 从词向量文本文件 word2vec 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# 成为 ngrams 的新词组，并在 word2vec 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
def wordVec(word, word2vec, min_n = 1, max_n = 3):
    # 确认词向量维度
    word_size = word2vec.wv.syn0[0].shape[0]

    # 如果在词典之中，直接返回词向量
    if word in word2vec.wv.vocab.keys():
        return word2vec[word]
    else:
        # 计算word的ngrams词组
        ngrams = compute_ngrams(word, min_n, max_n)
        # 不在词典的情况下
        word_vec = numpy.zeros(word_size, dtype=numpy.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in word2vec.wv.vocab.keys():
                word_vec += word2vec[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += word2vec[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            # 不抛出异常，而是打印提示，并返回0向量。
            print(KeyError('all ngrams for word %s absent from model' % word))
            return word_vec

In [ ]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self

In [ ]:
def isNan(a):
    return a != a

class RatingData(data.Dataset):
    def __init__(self, path, word2index, max_row = -1, trainTestRate = 0.85, isTrain = True):
        self.token_list = []
        self.label_list = []
        # self.token_positions = torch.tensor([i for i in range(100)])

        print('地址不应该包含 ratings.csv   path:',path) # 地址不应该包含 ratings.csv

        ratings_clean_filename = os.path.join(path, 'ratings_clean_4_HAN.csv')
        ratings_filename = os.path.join(path, 'ratings.csv')
        if os.path.isfile(ratings_clean_filename):
            clean_pd = pd.read_csv(ratings_clean_filename)
        else:
            print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
            ratings_pd = pd.read_csv(ratings_filename)
            print('开始生成缓存文件%s'%(ratings_clean_filename))
            clean_pd = pd.DataFrame({
                'userId':[],
                'restId':[],
                'rating':[],
                'comment':[],
            })
            nonRatingCount = 0
            for i, row in ratings_pd.iterrows():
                if max_row != -1 and i > max_row:
                    break
                if not isinstance(row['comment'], str) or row['comment'] == '':
                    # print(i + 1, row['comment'])
                    nonRatingCount += 1
                    continue
                r0 = row['rating']
                r1 = row['rating_env']
                r2 = row['rating_flavor']
                r3 = row['rating_service']
                if r0 == '' or isNan(r0): r0 = 0 # 假设总评分为 0 表示未评分
                if r1 == '' or isNan(r1): r1 = 3
                if r2 == '' or isNan(r2): r2 = 3
                if r3 == '' or isNan(r3): r3 = 3
                r0 = round(r0 * 0.5 + (r1 + r2 + r3) * 0.1666666)
                if i % 10000 == 9999:
                    print(i + 1, r0)

                # token = tokenizer.encode(text=str(row['comment']), max_length=100, pad_to_max_length = True)
                words = thulac.cut(row['comment'], text=True)
                # 0 在词向量集中是‘，’，换个词向量集可能表示其他
                token = [ word2index[words[i]] if i < len(words) and words[i] in word2index else 0 for i in range(100)] 
                
                newRow = DotDict()
                newRow.userId = [row['userId']]
                newRow.restId = [row['restId']]
                newRow.rating = [r0]
                newRow.comment = [json.dumps(token)]

                clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
            print('空的评论数量： %d'%(nonRatingCount))
            clean_pd.to_csv(ratings_clean_filename)

        # 读取
        if isTrain:
            temp_pd = clean_pd[ : int(len(clean_pd) * trainTestRate)]
        else:
            temp_pd = clean_pd[int(len(clean_pd) * trainTestRate) : ]

        for i, row in temp_pd.iterrows():
            if max_row != -1 and i > max_row:
                break

            self.label_list.append(torch.tensor(row['rating']).long())
            self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['comment']) ) ).long())

    def __getitem__(self, index):
        return self.token_list[index], self.label_list[index]#, self.token_positions

    def __len__(self):
        return len(self.label_list)
    

In [ ]:
ratingData = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             max_row= 100000,
                             isTrain=True,
                             )
trainLoader = torch.utils.data.DataLoader(dataset=ratingData,
                                          batch_size=10,
                                          shuffle = True,
                                          # num_workers = 0,
                                          )
ratingData2 = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             max_row= 100000,
                             isTrain=False,
                             )
testLoader = torch.utils.data.DataLoader(dataset=ratingData2,
                                          batch_size=10,
                                          shuffle = True,
                                          # num_workers = 0,
                                          )
print(len(ratingData.label_list))
print(len(ratingData2.label_list))